In [ ]:
from src.utils.loadData import load_data, TRAIN_DATA_POINTS, TEST_DATA_POINTS, N_CLASSES
from src.utils.processing import generate_class_matrix, generate_test_class_matrix

train_data = load_data("data/ae.train", num_data_points=TRAIN_DATA_POINTS)
test_data = load_data("data/ae.test", num_data_points=TEST_DATA_POINTS)

train_class_matrix = generate_class_matrix(TRAIN_DATA_POINTS, N_CLASSES)
test_class_matrix = generate_test_class_matrix(TEST_DATA_POINTS, N_CLASSES)

The following code displays a random data point.

In [ ]:
from src.utils.visualizeData import visualize_data_point
import numpy as np

data_idx = np.random.randint(0, TRAIN_DATA_POINTS)

print(f"Showing data point {data_idx}.")
visualize_data_point(train_data[data_idx])

Visualizing the pattern mean of the training data. The tail of the signal is less smooth, because the number of data points that reach that lenght taper off.

In [ ]:
from src.utils.processing import get_pattern_mean

train_pattern_mean = get_pattern_mean(train_data)
visualize_data_point(train_pattern_mean)

The distribution of signal lengths in the training data

In [ ]:
from src.utils.processing import get_time_steps
from src.utils.visualizeData import visualize_data_lengths

time_steps = get_time_steps(train_data)
visualize_data_lengths(time_steps)

Visualizing the pattern means for all classes, to get an insight and feel for the different classes.

In [ ]:
from src.utils.visualizeData import visualize_class_means

visualize_class_means(train_data)

The first step of Principal Component Analysis (PCA) is to obtain the Principal Components (PCs) and the respective feature variances.

In [ ]:
from src.linear_regression.pca import normalize, SVD

train_data = normalize(train_data, train_pattern_mean)
principal_components, feature_variances = SVD(train_data)

Here we can see the variance distribution of the PCs. Notice that there are more features than data points. Therefore, the variance after the number of training data points (270) drops to 0.

In [ ]:
from src.utils.visualizeData import visualize_PC_variance

visualize_PC_variance(feature_variances, log=False)

Here we obtain the regression classifier.

In [ ]:
from src.linear_regression.pca import reduce_PCs, get_feature_vectors
from src.linear_regression.classifier import compute_regression_classifier

reduced_PCs = reduce_PCs(feature_variances, principal_components, wanted_variance=99)
feature_vectors = get_feature_vectors(train_data, reduced_PCs)
regression_classifier = compute_regression_classifier(feature_vectors, train_class_matrix)

In [ ]:
from src.linear_regression.classifier import compute_MSE, compute_mismatch

train_mse = compute_MSE(regression_classifier, feature_vectors, train_class_matrix)
train_mismatch = compute_mismatch(regression_classifier, feature_vectors, train_class_matrix)

print("MSE:\t\t{:.2f}".format(train_mse))
print("Mismatch:\t{:.2f}%".format(train_mismatch * 100))

Here we first obtain the test feature vectors, and then compute the accuracy of our regression classifier.

In [ ]:
test_data = normalize(test_data, train_pattern_mean)
test_feature_vectors = get_feature_vectors(test_data, reduced_PCs)

test_mse = compute_MSE(regression_classifier, test_feature_vectors, test_class_matrix)
test_mismatch = compute_mismatch(regression_classifier, test_feature_vectors, test_class_matrix)

print("MSE:\t\t{:.2f}".format(test_mse))
print("Mismatch:\t{:.2f}%".format(test_mismatch * 100))

We now move on to our k-nearest neighbors (knn) approach

The following block of code is run to perform a hyperparameter search on the number of nearest neighbors.

In [ ]:
from src.knn.knn_pipeline import knn_hyperparameter_search

k_to_search = range(1, 11)
path = "search_results.txt"

# leave-one-out cross validation
knn_hyperparameter_search(
    data=train_data, 
    labels=train_class_matrix, 
    k_to_search=k_to_search, 
    k_folds=TRAIN_DATA_POINTS, 
    results_location=path)

The results of the hyperparameter search for 1 through 10 nearest neighbors are shown by the function in the next block. 
The search is done for two different local distance measures in the Dynamic Time Warping (DTW) procedure. The local distance measures are the absolute distance and the squared distance.

In [ ]:
from src.utils.visualizeData import show_search_results

show_search_results()

The following code performs the knn classification on the test data and returns the accuracy. It may take a few minutes to run.

In [ ]:
from src.knn.knn_pipeline import knn_accuracy

k_nn = 3
accuracy = knn_accuracy(k_nn, test_data, test_class_matrix, train_data, train_class_matrix)
knn_mismatch = 100 - accuracy
print("Mismatch:\t{:.2f}%".format(knn_mismatch * 100))